## Importando dependências

In [ ]:
!python -m pip install scikit-learn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Importando dados

In [ ]:
df_main = pd.read_csv('data/IM_230626_semNP.csv')

## Carteira Serviços

A métrica utilizada será a taxa de inadimplência, calculada pela carteira de direitos de aquisição inadimplentes dividido pelo patrimônio líquido

In [ ]:
#Cálculo da taxa de inadimplência em df_main, onde a taxa é definida pela divisão da coluna "Carteira_Direitos_Aquisicao_Inadimplentes" pela coluna "Patrimonio_Liquido". 
df_main = df_main.dropna(subset=["Patrimonio_Liquido"])
inadimplentes = df_main["Carteira_Direitos_Aquisicao_Inadimplentes"]
patrimonio_liquido = df_main["Patrimonio_Liquido"]
taxa_inadimplencia = inadimplentes / patrimonio_liquido

#Adição da taxa como uma nova coluna no DataFrame
df_main = df_main.assign(taxa_inadimplencia_series=taxa_inadimplencia)

In [ ]:
fundos = df_main[
    (df_main['taxa_inadimplencia_series'] != 0) &
    df_main['taxa_inadimplencia_series'].notna()
]

list(fundos['Nome_Fundo'].unique())

Selecionou-se os fundos cuja taxa de inadimplência fosse válida, ou seja, apenas aqueles que a taxa de inadimplência fosse diferente de 0 e NaN

In [ ]:
#Escolha das carteiras que serão utilizadas
carteiras = [
    'Carteira',
    'Carteira_Industrial',
    'Carteira_Mercado_Imobiliario',
    'Carteira_Comercial_Total',
    'Carteira_Comercial',
    'Carteira_Comercial_Varejo',
    'Carteira_Arrendamento_Mercantil',
    'Carteira_Servicos_Total',
    'Carteira_Servicos',
    'Carteira_Servicos_Publicos',
    'Carteira_Servicos_Educacionais',
    'Carteira_Entretenimento',
    'Carteira_Agronegocio',
    'Carteira_Financeiro',
    'Carteira_Credito_Pessoal_Consignado',
    'Carteira_Credito_Corporativo',
    'Carteira_Middle_Market',
    'Carteira_Veiculos',
    'Carteira_Imobiliaria_Empresarial',
    'Carteira_Imobiliaria_Residencial',
    'Carteira_Outros_Financeiro',
    'Carteira_Cartao_Credito',
    'Carteira_Factoring',
    'Carteira_Factoring_Pessoal',
    'Carteira_Factoring_Corporativo',
    'Carteira_Setor_Publico',
    'Carteira_Precatorios',
    'Carteira_Creditos_Tributarios',
    'Carteira_Royalties',
    'Carteira_Outros_Setor_Publico',
    'Carteira_Acoes_Judiciais',
    'Carteira_Propriedade_Intelectual',
]

#Cálculo da soma das carteiras agrupadas pela taxa de inadimplência e 
# plot de um gráfico de barras para mostrar as taxas totais de inadimplência por carteira
soma_carteiras = df_main.groupby('taxa_inadimplencia_series')[carteiras].sum()
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=soma_carteiras, palette='magma')
plt.title('Taxas de Inadimplência por Carteira')
plt.ylabel('Taxa total de inadimplência')
plt.xlabel('Carteira')
plt.xticks(rotation=45, ha='right')

plt.grid(True)
plt.tight_layout()

plt.show()

O gráfico foi plotado com todos os tipos de carteira, avaliando a taxa de inadimplência de cada segmento.

In [ ]:
soma_carteiras.head(5)

## K-means

O K-means é um algoritmo de agrupamento de dados que divide um conjunto de pontos em grupos (clusters) com base em suas características similares, buscando minimizar a variância dentro de cada grupo.

In [ ]:
from sklearn.cluster import KMeans

#Análise de cluster (K-means) com três clusters
carteira_servicos = 'Carteira_Servicos'

fundos = df_main[df_main[carteira_servicos].notna()]

X = fundos[[carteira_servicos]]

kmeans = KMeans(n_clusters=3)
fundos['grupo'] = kmeans.fit_predict(X)

#Criação de gráfico de dispersão para visualizar os grupos resultantes
cores = ['#FFA500', '#00CED1', '#00008B']
fig, ax = plt.subplots(figsize=(12, 8))

for grupo in range(3):
    grupo_df = fundos[fundos['grupo'] == grupo]
    ax.scatter(grupo_df.index, grupo_df[carteira_servicos],
               color=cores[grupo], label=f'Cluster {grupo}', s=100, alpha=0.7)

ax.set_title(f'{carteira_servicos}', fontsize=16)
ax.set_xlabel('.', fontsize=14)
ax.set_ylabel('Inadimplência', fontsize=14)
ax.set_xticks([])
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

ax.set_facecolor('white')

plt.tight_layout()
plt.show()

In [ ]:
#Filtrar o DataFrame "fundos" para selecionar cluster 1 e classificar os dados com base na coluna "taxa_inadimplencia_series" em ordem decrescente
cluster1 = fundos[fundos['grupo'] == 1]

cluster1 = cluster1.sort_values(by='taxa_inadimplencia_series', ascending=False)

fundos_inadimplencia = cluster1.head(10)

#Imprime informações relevantes sobre os fundos
for index, row in fundos_inadimplencia.iterrows():
    print(f'Nome do Fundo: {row["Nome_Fundo"]}')
    print(f'CNPJ do Fundo: {row["CNPJ"]}')
    print(f'Taxa de Inadimplência: {row["taxa_inadimplencia_series"]}')
    print(f'Data de competência: {row["Data_Competencia"]}')
    print('-' * 30)

Mostra as maiores taxas de inadimplência do cluster 1, ou seja do cluster que agrupa as maiores taxas de inadimplência dos dados.

# Escolha do modelo

Ao analisar a separação dos clusters da carteira comercial entre os algoritmos, o modelo escolhido foi o K-Means, pelo fato da sua separação ter sido uma das mais lógicas em relação ao nível de inadimplência, apesar do modelo não classificar os fundos relacionando-os diretamente com as features. Logo, é possível concluir que nenhum dos modelos candidatos apresentou uma separação muito clara ou significativa entre os clusters. Enquanto o K-Means e o Mini Batch K-Means realizaram uma separação em 3 partes proporcionais em relação ao target, o Mean Shift e o DBSCAN acabaram formando um cluster muito grande por conta de uma densidade de dados muito alta em determina, resultando em uma divisão muito desigual e também pouco significativa.